In [ ]:
# Required libraries
import numpy as np
import tensorflow as tf
import pickle
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score
import os
import tensorflow_model_optimization as tfmot
import tempfile

# Global variables
subj = 'subject1'
path = './data'

# Function definitions

def evaluate_tflite_model(tflite_model, X_test, y_test):
    """Evaluate the accuracy of a TFLite model."""
    interpreter = tf.lite.Interpreter(model_content=tflite_model)
    interpreter.allocate_tensors()
    input_index = interpreter.get_input_details()[0]["index"]
    output_index = interpreter.get_output_details()[0]["index"]

    predictions = []
    for test_example in X_test:
        test_example = np.expand_dims(test_example, axis=0).astype(np.float32)
        interpreter.set_tensor(input_index, test_example)
        interpreter.invoke()
        prediction = interpreter.get_tensor(output_index)
        predictions.append(prediction[0])

    y_pred_tflite = np.argmax(predictions, axis=1)
    accuracy = accuracy_score(y_test, y_pred_tflite)
    return accuracy

def representative_data_gen():
    """Generator function for representative data required for TFLite conversion."""
    for input_value in tf.data.Dataset.from_tensor_slices(X_train.astype(np.float32)).batch(1).take(100):
        yield [input_value]

def divide_samples(data, states, division):
    """Divide the data samples for more granular analysis."""
    divisions = division
    new_data_shape = (data.shape[0] * divisions, int(data.shape[1] / divisions))
    new_data = data.reshape(new_data_shape)
    new_states = np.repeat(states, divisions)
    return new_data, new_states

def get_gzipped_model_size(file):
    """Calculate gzipped model size for comparison."""
    import gzip
    import shutil
    _, zipped_file = tempfile.mkstemp('.zip')
    with open(file, 'rb') as f_in, gzip.open(zipped_file, 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)
    return os.path.getsize(zipped_file)


Data = np.load(f"{path}/subset_data_{subj}.npy")
with open(f"{path}/subset_metadata_{subj}.pkl", 'rb') as f:
    Data_metadata = pickle.load(f)

# Define labels for Data
labels_Data = np.array([0 if state == 'MA' else 1 if state == 'awake' else 2 for state in Data_metadata['GT']])
division = 10
subset_da, subset_sts = divide_samples(Data, labels_Data, division)

# Normalize Data
scaler = MinMaxScaler(feature_range=(-1, 1))
data_scaled = scaler.fit_transform(subset_da)
data_scaled_int16 = data_scaled.astype(np.float16)

# Encode states
encoder = LabelEncoder()
states_encoded = encoder.fit_transform(subset_sts)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data_scaled_int16, states_encoded, test_size=0.2, random_state=42)

model = Sequential([
    Dense(8, activation='relu', input_shape=(subset_da.shape[1],)),
    Dense(4, activation='relu'),
    Dense(len(np.unique(states_encoded)), activation='softmax')
])
model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=120, batch_size=64)

y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
original_accuracy = accuracy_score(y_test, y_pred_classes)

pruning_params = {
    'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(
        initial_sparsity=0.85, final_sparsity=0.98, begin_step=0, end_step=np.ceil(X_train.shape[0] * 0.9 / 128).astype(np.int32) * 2
    )
}
model_for_pruning = tfmot.sparsity.keras.prune_low_magnitude(model, **pruning_params)
model_for_pruning.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
model_for_pruning.fit(X_train, y_train, batch_size=128, epochs=2, validation_split=0.1, callbacks=[tfmot.sparsity.keras.UpdatePruningStep()])

# Evaluate and compare the pruned model
model_for_export = tfmot.sparsity.keras.strip_pruning(model_for_pruning)
model_for_export.save("pruned_model.h5")

converter = tf.lite.TFLiteConverter.from_keras_model(model_for_export)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.float16]
quantized_and_pruned_tflite_model = converter.convert()

# Evaluate TFLite model
quantized_pruned_accuracy = evaluate_tflite_model(quantized_and_pruned_tflite_model, X_test, y_test)

# Print comparisons and model sizes

print("Original Model Accuracy: {:.2f}%".format(original_accuracy * 100))
print("Quantized and Pruned Model Accuracy: {:.2f}%".format(quantized_pruned_accuracy * 100))

# Calculate and display model sizes
model.save('myFullModel.h5')
original_model_size = get_gzipped_model_size('myFullModel.h5')
quantized_pruned_model_size = get_gzipped_model_size("pruned_model.h5")
print("Size of gzipped original Keras model: %.2f Kbytes" % (original_model_size / 1024))
print("Size of gzipped pruned and quantized TFLite model: %.2f Kbytes" % (quantized_pruned_model_size / 1024))


In [ ]:
import tensorflow as tf
import numpy as np
import os
from sklearn.metrics import accuracy_score

model.save("myFullModel.h5")

# Calculate the size of the original model
model_size2 = os.path.getsize("myFullModel.h5") / 1024  # size in KB
print("Model Size: {:.2f} KB".format(model_size2))

# Evaluate the original model
print("Evaluating the original model...")
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
original_accuracy = accuracy_score(y_test, y_pred_classes)
print("Original Model Accuracy: {:.2f}%".format(original_accuracy * 100))

# Convert the model to the TensorFlow Lite format without quantization
print("Converting the model to TensorFlow Lite format...")
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model2 = converter.convert()
print("Conversion to TensorFlow Lite completed.")

# Save the non-quantized model to disk
print("Saving TensorFlow Lite model to disk...")
with open("model2.tflite", "wb") as f:
    f.write(tflite_model2)

# Apply post-training quantization for float16
print("Applying post-training float16 quantization...")
converter.optimizations = [tf.lite.Optimize.DEFAULT]
#converter.target_spec.supported_types = [tf.float16]

# Convert the model with float16 quantization
print("Converting the model with float16 quantization...")
tflite_quant_model2 = converter.convert()
print("Float16 quantized model conversion completed.")

# Save the quantized model to disk
print("Saving float16 quantized model to disk...")
with open("Fullmodel_quantized2_float16.tflite", "wb") as f:
    f.write(tflite_quant_model2)

print("Evaluating both original and quantized TFLite models...")
original_tflite_accuracy2 = evaluate_tflite_model(tflite_model2, X_test, y_test)
quantized_tflite_accuracy2 = evaluate_tflite_model(tflite_quant_model2, X_test, y_test)

print("TFLite Model Accuracy: {:.2f}%".format(original_tflite_accuracy2 * 100))
print("Quantized TFLite Model Accuracy: {:.2f}%".format(quantized_tflite_accuracy2 * 100))

# Check the size of the models in kilobytes
print("Calculating model sizes...")
model_size2 = os.path.getsize("model2.tflite") / 1024
quant_model_size2 = os.path.getsize("Fullmodel_quantized2_float16.tflite") / 1024
print("Model Size 2: {:.2f} KB".format(model_size2))
print("Quantized Model Size2: {:.2f} KB".format(quant_model_size2))


## Plots
Note that the accuracy in this case of the models that we will deploy are not evaluated 'on device'. The values, however, do not vary much. 

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def plot_model_metrics(state_of_the_art_accuracy,soa_size,
                       original_accuracy, original_model_size, 
                       quantized_accuracy, quantized_model_size, 
                       quantized_pruned_accuracy, quantized_pruned_model_size,):
    """
    Plot the accuracies and sizes of the original, quantized, and quantized-pruned models in a single grouped bar chart.

    Parameters:
    original_accuracy (float): Accuracy of the original model.
    original_model_size (int): Size of the original model.
    quantized_accuracy (float): Accuracy of the quantized model.
    quantized_model_size (int): Size of the quantized model.
    quantized_pruned_accuracy (float): Accuracy of the quantized and pruned model.
    quantized_pruned_model_size (int): Size of the quantized and pruned model.
    """

    labels = ['State of the Art (1D Conv + VAE)','Original', 'Quantized', 'Quantized & Pruned']
    accuracies = [state_of_the_art_accuracy, original_accuracy * 100,quantized_pruned_accuracy * 100, quantized_accuracy * 100] #note that in this plot, the accuracy is not calculated on-device
    sizes = [soa_size, original_model_size / 1024, quantized_pruned_model_size / 1024, quantized_model_size ] 

    # Creating bar positions
    x = np.arange(len(labels))
    width = 0.35

    fig, ax1 = plt.subplots(figsize=(12, 6))

    # Plotting Accuracies
    rects1 = ax1.bar(x - width/2, accuracies, width, label='Accuracy (%)', color='blue')

    # Creating a second y-axis for sizes
    ax2 = ax1.twinx()
    rects2 = ax2.bar(x + width/2, sizes, width, label='Size (Kbytes)', color='green')

    # Adding labels, title, and custom x-axis tick labels
    ax1.set_xlabel('Model Type')
    ax1.set_ylabel('Accuracy (%)', color='blue')
    ax2.set_ylabel('Size (Kbytes)', color='green')
    ax1.set_title('Model Accuracies and Sizes')
    ax1.set_xticks(x)
    ax1.set_xticklabels(labels)

    ax1.tick_params(axis='y', labelcolor='blue')
    ax2.tick_params(axis='y', labelcolor='green')
    fig.tight_layout()
    fig.legend(loc='upper right')

    plt.show()

soa_size = 250
state_of_the_art_accuracy = 92
#note that state_of_the_art_accuracy and soa_size are defined by previous work, 
#refs: marin-llobet et al. 2023; marin-llobet & manasanch et al. 2023; manasanch et al. 2024

plot_model_metrics(

    state_of_the_art_accuracy,
    soa_size,


    original_accuracy, 
    original_model_size, 
    
    quantized_tflite_accuracy2, 
    quant_model_size2, 

    quantized_pruned_accuracy, 
    quantized_pruned_model_size,
)

## To obtain dummy data to test on HW 
Please, select segments of 200 time series values of one 

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score

num_segments = 2
segment_length = 25
total_length = len(y_pred_classes)
segment_size = total_length // num_segments

for i in range(num_segments):
    # Calculate start and end index for the segment
    start_idx = i * segment_size
    end_idx = start_idx + segment_length

    # Ensure we don't go out of bounds
    end_idx = min(end_idx, total_length)

    # Extract the segment from X_test and y_pred_classes
    X_test_segment = X_test[start_idx:end_idx]
    y_pred_classes_segment = y_pred_classes[start_idx:end_idx]

    # Print the segment and corresponding predictions in Arduino-friendly format
    print(f"// Segment {i+1}:")
    print(f"float X_test_segment_{i+1}[{segment_length}][{X_test_segment.shape[1]}] = {{")
    for sample in X_test_segment:
        print(f"  {{{' ,'.join(map(str, sample))}}},")
    print("};")
    print(f"int y_pred_classes_segment_{i+1}[{segment_length}] = {{{' ,'.join(map(str, y_pred_classes_segment))}}};")
    print("\n")

## Pretrained model conversion from Tflite to CC
To convert Tflite models into CC (for deployment)


In [ ]:
MODEL_TFLITE = 'model2.tflite' #chose the model of your choice, i.e., model2 / quantized_float16 / etc.
MODEL_TFLITE_MICRO = 'model2.cc'
!xxd -i {MODEL_TFLITE} > {MODEL_TFLITE_MICRO}
REPLACE_TEXT = MODEL_TFLITE.replace('/', '_').replace('.', '_')
!sed -i 's/'{REPLACE_TEXT}'/g_model/g' {MODEL_TFLITE_MICRO}

In [ ]:
!cat {MODEL_TFLITE_MICRO}